In [166]:
from environs import Env

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationTokenBufferMemory
from langchain.chains import ConversationChain, LLMChain
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts.prompt import PromptTemplate
from typing import List, Dict, Any

In [167]:
class CustomConversationTokenBufferMemory(ConversationTokenBufferMemory):
    def clear(self):
        super().clear()
        buffer = self.chat_memory.messages
        curr_buffer_length = self.llm.get_num_tokens_from_messages(buffer)
        while curr_buffer_length > self.max_token_limit:
            buffer.pop(0)
            curr_buffer_length = self.llm.get_num_tokens_from_messages(buffer)
    
    extra_variables:List[str] = []

    @property
    def memory_variables(self) -> List[str]:
        """Will always return list of memory variables."""
        return [self.memory_key] + self.extra_variables

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Return buffer with history and extra variables"""
        d = super().load_memory_variables(inputs)
        d.update({k:inputs.get(k) for k in self.extra_variables})        
        return d

In [2]:
# import env config file
env = Env()
env.read_env('../.env', recurse=False)

True

### Чтение системного промпта - инструкции с файла

In [179]:
with open('../docs/system_prompt.txt', 'r') as f:
    system_prompt = f.read()

### Чтение скрипта продаж

In [178]:
with open('../docs/train_script.txt', 'r') as f:
    scripts_text = f.read()

### Цепь общения с сохранением истории

In [473]:
def get_conversation_chain_history(prompt_template, verbose=False):
    llm = ChatOpenAI(openai_api_key=env("OPENAI_TOKEN"), temperature=0.7)
    # conversation = LLMChain(llm=llm, prompt=prompt_template)
    conversation = ConversationChain(
    llm=llm, verbose=verbose, prompt=prompt_template, memory=CustomConversationTokenBufferMemory(max_token_limit=200,llm=llm, human_prefix="Клиент", ai_prefix="AI ассистент", extra_variables=["scripts_text", "system_prompt"]))
    return conversation

### Цепь общения с сохранением последнего вопроса-ответа

In [129]:
def get_conversation_chain_window(prompt_template, verbose=False):
    llm = ChatOpenAI(openai_api_key=env("OPENAI_TOKEN"), temperature=0.7)
    conversation = ConversationChain(
    llm=llm, verbose=verbose, prompt=prompt_template, memory=ConversationBufferWindowMemory(k=1, human_prefix="Клиент", ai_prefix="AI ассистент")
)
    return conversation

### Цепь общения с сохранением контекста истории общения

In [140]:
def get_conversation_chain_summary(prompt_template, verbose=False):
    llm = ChatOpenAI(openai_api_key=env("OPENAI_TOKEN"), temperature=0.7)
    conversation = ConversationChain(
    llm=llm, verbose=verbose, prompt=prompt_template, memory=ConversationSummaryMemory(llm=llm)
)
    return conversation

### Сборка промпта

In [173]:
template = """{system_prompt}\n
Скрипт продаж:\n
{scripts_text}\n


История общения с текущим клиентом: \n
{history}\n
Вопрос: \n
{input} \n
Ответ:\n

"""
prompt_template = PromptTemplate(
    input_variables= ["scripts_text", "system_prompt",  "history", "input"],
    template=template,
  
)

### Новая цепочка общения

In [474]:
# create conversation chain
state = get_conversation_chain_history(prompt_template, verbose=False)

In [477]:
state.invoke( input="Меня зовут Дима", verbose=False)['response']

'Приятно познакомиться, Дима! Чем я могу вам помочь?'

In [439]:
state.memory.save_context({"input": "hdadadadidadadaaaa"}, {"output": "whats up"})
state.memory.load_memory_variables({})

{'history': 'AI ассистент: whats up\nКлиент: hdadadadidadadaaaa\nAI ассистент: whats up\nКлиент: hdadadadidadadaaaa\nAI ассистент: whats up\nКлиент: hdadadadidadadaaaa\nAI ассистент: whats up\nКлиент: hdadadadidadadaaaa\nAI ассистент: whats up\nКлиент: hdadadadidadadaaaa\nAI ассистент: whats up\nКлиент: hdadadadidadadaaaa\nAI ассистент: whats up\nКлиент: hdadadadidadadaaaa\nAI ассистент: whats up\nКлиент: hdadadadidadadaaaa\nAI ассистент: whats up\nКлиент: hdadadadidadadaaaa\nAI ассистент: whats up',
 'scripts_text': None,
 'system_prompt': None}

In [440]:
id = 22
username = 'test'
memory = [item['content'] for item in state.memory.chat_memory.dict()['messages']]
inputs_memory = [item for item in memory[::2]]
outputs_memory = [item for item in memory[1::2]]
conversations = [{'id': id, 'username': 'username', 'inputs': inputs_memory, 'outputs': outputs_memory}]

In [445]:
import pickle
import os

pickle_filename = 'test.pkl'
with open(pickle_filename, "wb") as f:
    pickle.dump(conversations, f)
    print("Variable saved to pickle file.", type(conversations))

Variable saved to pickle file. <class 'list'>


In [466]:
import sqlite3

connection = sqlite3.connect('../seller_bot.db')
cursor = connection.cursor()        
# Извлекаем историю сообщений пользователя
cursor.execute('SELECT input, output FROM chats WHERE user_id = ? ORDER BY chat_id DESC LIMIT 10', (1, ))
user_history= cursor.fetchall()
user_history

[]

In [469]:
state.invoke(in)

TypeError: Chain.invoke() missing 1 required positional argument: 'input'

In [447]:
if os.path.exists(pickle_filename):
    with open(pickle_filename, "rb") as f:
        conversations_retr = pickle.load(f)
        print("Variable loaded from pickle file.")
conversations_retr

Variable loaded from pickle file.


[{'id': 22,
  'username': 'username',
  'inputs': ['whats up',
   'whats up',
   'whats up',
   'whats up',
   'whats up',
   'whats up',
   'whats up',
   'whats up',
   'whats up',
   'whats up'],
  'outputs': ['hdadadadidadadaaaa',
   'hdadadadidadadaaaa',
   'hdadadadidadadaaaa',
   'hdadadadidadadaaaa',
   'hdadadadidadadaaaa',
   'hdadadadidadadaaaa',
   'hdadadadidadadaaaa',
   'hdadadadidadadaaaa',
   'hdadadadidadadaaaa']}]

{'key': 'value'}

In [183]:
state.run(system_prompt=system_prompt, scripts_text=scripts_text, input="Меня зовут Дима")



> Entering new ConversationChain chain...
Prompt after formatting:
Ты чат-бот для продаж курсов по нейропсихологии от компании Университет НЛП Ты должен отвечать на вопросы лидов в телеграм-боте. 
Приветствовать, давать информацию по курсам, отправлять контакты для связи и ссылки на сайт и другие социальные сети. Также твоя задача задавать ему уточняющие вопросы.
Я дам тебе скрипты продаж, где приведены потенциальные вопросы лидов и идеальные ответы. 
Ты должен отвечать полагаясь на информацию в этом скрипте, придерживаясь этой манеры. 
А также всегда давая информацию о сайте и номере телефона. Не отвечай на вопросы не по теме. 
Твоя задача продать наши курсы. За каждый проданный курс ты получаешь 100$.
Обращайся к клиенту по имени. Если не знаешь его имени задавай ему вопросы и запомни.


Скрипт продаж:

Вопрос: Привет!
Ответ: Добрый день! 👋 Рады видеть вас в Университет НЛП. Как мы можем помочь вам сегодня?
Вопрос: Мне интересно узнать больше о курсах по нейропсихологии.
Ответ: У н

'Привет, Дима! Как я могу помочь тебе сегодня? Если у тебя есть вопросы о наших курсах по нейропсихологии или ты хочешь узнать больше информации, я готов помочь.'

In [196]:
state.memory.load_memory_variables({})

{'history': 'Клиент: Привет\nAI ассистент: Добрый день! 👋 Рады видеть вас в Университете НЛП. Как мы можем помочь вам сегодня?\nКлиент: Привет\nAI ассистент: Добрый день! 👋 Рады видеть вас в Университете НЛП. Как мы можем помочь вам сегодня?\nКлиент: Как меня зовут?\nAI ассистент: Извините, я не могу запоминать имена. Могу я помочь вам с чем-то еще?\nКлиент: Меня зовут Дима\nAI ассистент: Привет, Дима! Как я могу помочь тебе сегодня?\nКлиент: Меня зовут Дима\nAI ассистент: Привет, Дима! Как я могу помочь тебе сегодня?\nКлиент: Меня зовут Дима\nAI ассистент: Привет, Дима! Как я могу помочь тебе сегодня? Если у тебя есть вопросы о наших курсах по нейропсихологии или ты хочешь узнать больше информации, я готов помочь.',
 'scripts_text': None,
 'system_prompt': None}

In [197]:
state.memory.save_context({"input": "hi"}, {"output": "whats up"})

In [202]:
some = "Клиент: Привет\nAI ассистент: Добрый день! "

In [215]:
state.memory.chat_memory.dict()['messages']

[{'content': 'Привет',
  'additional_kwargs': {},
  'type': 'human',
  'example': False},
 {'content': 'Добрый день! 👋 Рады видеть вас в Университете НЛП. Как мы можем помочь вам сегодня?',
  'additional_kwargs': {},
  'type': 'ai',
  'example': False},
 {'content': 'Привет',
  'additional_kwargs': {},
  'type': 'human',
  'example': False},
 {'content': 'Добрый день! 👋 Рады видеть вас в Университете НЛП. Как мы можем помочь вам сегодня?',
  'additional_kwargs': {},
  'type': 'ai',
  'example': False},
 {'content': 'Как меня зовут?',
  'additional_kwargs': {},
  'type': 'human',
  'example': False},
 {'content': 'Извините, я не могу запоминать имена. Могу я помочь вам с чем-то еще?',
  'additional_kwargs': {},
  'type': 'ai',
  'example': False},
 {'content': 'Меня зовут Дима',
  'additional_kwargs': {},
  'type': 'human',
  'example': False},
 {'content': 'Привет, Дима! Как я могу помочь тебе сегодня?',
  'additional_kwargs': {},
  'type': 'ai',
  'example': False},
 {'content': 'Меня

In [216]:
memory = [item['content'] for item in state.memory.chat_memory.dict()['messages']]
inputs_memory = [item for item in memory[::2]]
outputs_memory = [item for item in memory[1::2]]

In [482]:
import re

In [527]:
filename = 'documents/systemprompt_12314:abcd.txt'

In [528]:
re.split('/|_|\.',filename)

['documents', 'systemprompt', '12314:abcd', 'txt']

In [28]:
import sqlite3

user_id = 400690372
connection = sqlite3.connect('../seller_bot.db')
cursor = connection.cursor()        
# Проверяем наличие пользователя
cursor.execute('UPDATE users SET isadmin = 0 WHERE user_id = ?', (user_id,))
connection.commit()
connection.close()
cursor

In [34]:
import os

os.path.exists('../docs/chat_history.xlsx')

True

In [190]:
import pickle

In [192]:
with open("test.pkl", "wb") as f:
    pickle.dump(state.memory, f)
    print("Variable saved to pickle file.")

TypeError: cannot pickle 'SSLContext' object

In [472]:
state.invoke( input="Меня зовут Дима", verbose=False)['response']



> Entering new ConversationChain chain...
Prompt after formatting:
None

Скрипт продаж:

None



История общения с текущим клиентом: 

Клиент: hdadadadidadadaaaa
AI ассистент: whats up
Клиент: hdadadadidadadaaaa
AI ассистент: whats up
Клиент: hdadadadidadadaaaa
AI ассистент: whats up
Клиент: hdadadadidadadaaaa
AI ассистент: whats up
Клиент: hdadadadidadadaaaa
AI ассистент: whats up
Клиент: Меня зовут Дима
AI ассистент: Приятно познакомиться, Дима! Как я могу вам помочь?
Клиент: Меня зовут Дима
AI ассистент: Приятно познакомиться, Дима! Как я могу вам помочь?

Вопрос: 

Меня зовут Дима 

Ответ:




> Finished chain.


'Приятно познакомиться, Дима! Как я могу вам помочь?'

### Проверка ответа

In [481]:
import re

In [128]:
input = "Как записатсья на курс"
state.run({
    "input": input   
})



> Entering new ConversationChain chain...
Prompt after formatting:
Ты чат-бот для продаж курсов по нейропсихологии от компании Университет НЛП Ты должен отвечать на вопросы лидов в телеграм-боте. 
Приветствовать, давать информацию по курсам, отправлять контакты для связи и ссылки на сайт и другие социальные сети. Также твоя задача задавать ему уточняющие вопросы.
Я дам тебе скрипты продаж, где приведены потенциальные вопросы лидов и идеальные ответы. 
Ты должен отвечать полагаясь на информацию в этом скрипте, придерживаясь этой манеры. 
А также всегда давая информацию о сайте и номере телефона. Не отвечай на вопросы не по теме. 
Твоя задача продать наши курсы. За каждый проданный курс ты получаешь 100$.
Обращайся к клиенту по имени. Если не знаешь его имени задавай ему вопросы и запомни.


Скрипт продаж:

Вопрос: Привет!
Ответ: Добрый день! 👋 Рады видеть вас в Университет НЛП. Как мы можем помочь вам сегодня?
Вопрос: Мне интересно узнать больше о курсах по нейропсихологии.
Ответ: У н

'Для записи на курс вам нужно перейти на наш сайт https://neuro-moscow-nlp.ru и выбрать интересующий вас курс. Там вы найдете форму регистрации и инструкции по оплате. Если у вас возникнут вопросы или вам понадобится помощь, вы всегда можете связаться с нами через контакт-форму на сайте или по телефону +123456789.'